In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import scipy.stats as stats
from pathlib import Path
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split, KFold

In [2]:
# options
pd.set_option('max_columns',100)
plt.style.use('fivethirtyeight')
warnings.simplefilter('ignore')
seed = 1

In [3]:
# Data dirctory
data_dir = Path('../data/')
data_file = data_dir / 'data_raw.csv'

# Data
# 0:AAR / 1:EAD / 2:ADR / 3:EDD는 고정  , 나머지는 순서 상관 없음
Data = pd.read_csv(data_file, index_col=0)

In [ ]:
Data

# Continuous Variables

In [ ]:
y = Data['AAR']
plt.figure(1); plt.title('Johnson SU')
sns.distplot(y, kde=False, fit=stats.johnsonsu)
plt.figure(2); plt.title('Normal')
sns.distplot(y, kde=False, fit=stats.norm)
plt.figure(3); plt.title('Log Normal')
sns.distplot(y, kde=False, fit=stats.lognorm)
plt.figure(4); plt.title('Exponential')
sns.distplot(y, kde=False, fit=stats.expon)
plt.figure(5); plt.title('Laplace')
sns.distplot(y, kde=False, fit=stats.laplace)
#plt.figure(6); plt.title('Poisson ')
#sns.distplot(y, kde=False, fit=stats.poisson)
plt.figure(7); plt.title('t')
sns.distplot(y, kde=False, fit=stats.t)


In [ ]:
Data.info()

In [ ]:
Data.describe()

In [ ]:
Data['AAR'].hist(bins = 50)

In [ ]:
Data['AAR'].quantile(np.arange(0, 1, .01))

In [ ]:
Data.groupby('AAR').mean()

In [ ]:
target_col = 'AAR'
pred_col = 'ADR'
num_cols = [x for x in Data.columns if Data[x].dtype in [np.int64, np.float64] and x != target_col]
print(num_cols)

In [ ]:
df.box_off_num.hist(bins=50)


In [ ]:
sns.pairplot(data=df, vars=num_cols + [target_col], size=5)


In [ ]:
Data.corr().style.background_gradient()


In [ ]:
sns.pairplot(data=df, vars=[pred_col, target_col], size=4, plot_kws={'alpha': .5})


In [ ]:
sns.pairplot(trn.sample(n=10000), vars=['u', 'dered_u'], hue='class', size=5)


In [ ]:
Data['WS_over20'] = 0
Data['WS_over20'][Data['WSPD']>20] =1

# Categorical Variables

In [ ]:
Data['AAR'].value_counts()

In [ ]:

# label, Oridinal Encoding = 각 category를 정수로 변환하는 것
Data_cat = Data.copy()
oe = OrdinalEncoder()
Data_cat[cat_cols] = oe.fit_transform(Data[cat_cols])

# nominal - categorical variables 사이에 상하관계가 X
# ordinal - 상하관계, 순서가 O


# Decision tree이외의 algo는 one-hot encoding을 해야함!!
# pd.get_dummies
# from sklearn.preprocessing import OneHotEncoder

In [ ]:
# Target Encoding
# 각 category를 해당 cateory 안의 요소들의 label의 평균값으로 변환
# -> overfitting에 취약 -> CV, smoothing과 같이 사용

# Data[['원하는 column', '타켓 칼럼 = AAR, ADR']].groupby('원하는 column').transform('mean')

Data[['원하는 column', '타켓 칼럼 = AAR, ADR']].groupby('원하는 column').transform('mean')

In [ ]:
# Frequency Encoding
# 각 category를 요소들의 빈도로 변환

# pd.DataFrame(Data['원하는 column']].value_counts())
# Data.groupby('원하는 column')[['타겟 칼럼']].transform('count')

In [ ]:
# Hashing Trick
# 각각의 범주를 Hash 함수에 적용한 값으로 변환 

# from sklearn.feature_extraction import FeatureHasher
# fh = FeatureHasher()
# X = hstack([Data[num_cols]] + [fh.fit_transform(df[col]) for col in cat_cols])

Prepare data

1. Checking for NaN values and removing constant features in the training data
2. Removing duplicated columns
3. Drop Sparse Data

Add Features<br>
1. Sumzeros and Sumvalues 
2. Other Aggregates
3. K-Means 
4. PCA : Principal component analysis 



In [ ]:
# Feature Scaling

# decision tree류의 알고리즘은 Scaling(standardization, normalization)이 큰 의미 X


"""
from sklearn.preprocessing import StandardScaler, MinMaxScaler
scaler = StandardScaler()
X = scaler.fit_transform(df[num_cols])
lr = LinearRegression()
lr.fit(X, np.log1p(df[target_col]))
df[pred_col] = np.expm1(lr.predict(X))

scaler = MinMaxScaler()
X = scaler.fit_transform(df[num_cols])
lr = LinearRegression()
lr.fit(X, np.log1p(df[target_col]))
df[pred_col] = np.expm1(lr.predict(X))
"""

In [ ]:
#Binning

#어떤 feature를 n개의 그룹으로 나누고, 그것을 새로운 categorical data로 넣는 것

"""
df['time_bin'] = pd.qcut(df['time'], 4, labels=False)
sns.pairplot(data=df, vars=['time', 'time_bin'], size=4, plot_kws={'alpha': .5})


X = pd.concat([df[num_cols], pd.get_dummies(pd.qcut(df['time'], 4, labels=False))], axis=1)    
# get_dummies 는 one-hot encoding해주는 것(decision tree 계열을 안 하는게 보통 더 좋은 결과를 냄)

lr = LinearRegression()
lr.fit(X, np.log1p(df[target_col]))
df[pred_col] = np.expm1(lr.predict(X))
"""

In [ ]:
# Polynomial Regression

# 연속적인 몇개의 feature들을 조합해서 새로운 feature를 만드는 것 (overfitting 위험 O)

"""
from sklearn.preprocessing import PolynomialFeatures
poly = PolynomialFeatures(2)        # 2차(x^2, x1 * x2 등) 까지만 만들겠다
X = poly.fit_transform(df[num_cols])

lr = LinearRegression()
lr.fit(X, np.log1p(df[target_col]))
df[pred_col] = np.expm1(lr.predict(X))
"""

In [ ]:
# Checking for NaN values and removing constant features in the training data
# Removing duplicated columns
# Drop Sparse Data

### Add Features
# Sumzeros and Sumvalues
# Other Aggregates
# K-Means
# PCA : Principal component analysis



# Save

In [4]:
# 필요없는 것을 버리기
Data_temp = Data.drop('TMP', axis=1)
Data_temp = Data_temp.drop('TD', axis=1)
Data_temp = Data_temp.drop('HM', axis=1)
Data_temp = Data_temp.drop('PS', axis=1)
Data_temp = Data_temp.drop('PA', axis=1)

#고층바람 너무 높은 고도는 뺴자 
Data_temp = Data_temp.drop('WD_400', axis=1)
Data_temp = Data_temp.drop('WD_500', axis=1)
Data_temp = Data_temp.drop('WD_700', axis=1)
Data_temp = Data_temp.drop('WS_400', axis=1)
Data_temp = Data_temp.drop('WS_500', axis=1)
Data_temp = Data_temp.drop('WS_700', axis=1)

In [5]:
# drop TAF
for i in range(6,30,6):
    Data_temp = Data_temp.drop(f'WDIR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WSPD_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WG_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'VIS_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'WC_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_1LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_1LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_2LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_2LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'CLA_3LYR_t{i}', axis=1)
    Data_temp = Data_temp.drop(f'BASE_3LYR_t{i}', axis=1)

In [6]:
# 각 시간에 맞는 TAF로 나누기
taf6 = [12,18,24]
taf12 = [6,18,24]
taf18 = [6,12,24]
taf24 = [6,12,18]
    
# 각 시간에 맞는 taf 넣기
data_taf = {}
for i in range(6,30,6):
    data_taf[f'Data_{i}'] = Data_temp    
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WDIR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WSPD_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WG_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'VIS_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'WC_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_1LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_1LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_2LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_2LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'CLA_3LYR_t{i}'])
    data_taf[f'Data_{i}'] = data_taf[f'Data_{i}'].join(Data[f'BASE_3LYR_t{i}'])
    
Data_6 = data_taf['Data_6']
Data_12 = data_taf['Data_12']
Data_18 = data_taf['Data_18']
Data_24 = data_taf['Data_24']

In [7]:
# save total data for Jupyter notebook
Data.to_csv('..\\data\\data.csv')
Data_6.to_csv('..\\data\\data_6.csv')
Data_12.to_csv('..\\data\\data_12.csv')
Data_18.to_csv('..\\data\\data_18.csv')
Data_24.to_csv('..\\data\\data_24.csv')

In [8]:
# save data for .py

column_name_a = ['taf', 'label', 'EAD', 'EDD', 'year', 'month', 'day', 'hour', 'DayName', 'Arpt_cond',
               'P_Airp', 'P_AAR', 'P_ADR', 'Arrival_remainder', 'Departure_remainder',
               'WD_850', 'WD_925', 'WD_1000', 'WS_850', 'WS_925', 'WS_1000',
               'WD', 'WSPD', 'WS_GST', 'VIS', 'WC', 'RN', 
               'CA_TOT', 'CLA_1LYR', 'BASE_1LYR','CLA_2LYR', 'BASE_2LYR', 'CLA_3LYR', 'BASE_3LYR', 'CLA_4LYR','BASE_4LYR', 'RVR',
               'WDIR_t', 'WSPD_t', 'WG_t', 'VIS_t', 'WC_t',
               'CLA_1LYR_t', 'BASE_1LYR_t', 'CLA_2LYR_t', 'BASE_2LYR_t', 'CLA_3LYR_t', 'BASE_3LYR_t']
column_name_d = ['taf', 'EAD', 'label', 'EDD', 'year', 'month', 'day', 'hour', 'DayName', 'Arpt_cond',
               'P_Airp', 'P_AAR', 'P_ADR', 'Arrival_remainder', 'Departure_remainder',
               'WD_850', 'WD_925', 'WD_1000', 'WS_850', 'WS_925', 'WS_1000',
               'WD', 'WSPD', 'WS_GST', 'VIS', 'WC', 'RN', 
               'CA_TOT', 'CLA_1LYR', 'BASE_1LYR','CLA_2LYR', 'BASE_2LYR', 'CLA_3LYR', 'BASE_3LYR', 'CLA_4LYR','BASE_4LYR', 'RVR',
               'WDIR_t', 'WSPD_t', 'WG_t', 'VIS_t', 'WC_t',
               'CLA_1LYR_t', 'BASE_1LYR_t', 'CLA_2LYR_t', 'BASE_2LYR_t', 'CLA_3LYR_t', 'BASE_3LYR_t']
arrival_train = pd.DataFrame([], columns = column_name_a)
arrival_test = pd.DataFrame([], columns = column_name_a)
departure_train = pd.DataFrame([], columns = column_name_d)
departure_test = pd.DataFrame([], columns = column_name_d)

In [9]:
for i in range(6,30,6):

    data_name = 'Data_' + str(i) + '.csv'            
    data_file = data_dir / data_name
    Data_raw = pd.read_csv(data_file, index_col=0)
    Data_m = pd.read_csv(data_file, index_col=0)
    Data_d = Data_m.drop('AAR', axis=1)
    Data_a = Data_m.drop('ADR', axis=1)

    # Arrival
    train_a = Data_a
    train_a.rename(columns = {'AAR' : 'label'}, inplace = True)

    # Departure
    train_d = Data_d
    train_d.rename(columns = {'ADR' : 'label'}, inplace = True)

    # temp dataframe
    temp_train_a = pd.DataFrame({'taf' : i * np.ones(len(train_a))}).join(train_a)
    temp_train_a.columns = column_name_a
    temp_train_d = pd.DataFrame({'taf' : i * np.ones(len(train_d))}).join(train_d)
    temp_train_d.columns = column_name_d


    # append
    arrival_train = arrival_train.append(temp_train_a)
    departure_train = departure_train.append(temp_train_d)

In [10]:
# Save
arrival_train.reset_index(drop=True).to_csv('..\\input\\arrival_train.csv')
departure_train.reset_index(drop=True).to_csv('..\\input\\departure_train.csv')